In [2]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from dash import Dash,dcc,html,Input,Output,callback

In [37]:
data ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
csv = pd.read_csv(data)
df = pd.DataFrame(csv)
df = df.drop(columns=['Unnamed: 0'])


In [39]:
df.head(2)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [52]:
success = df.query('`class` == 1')

In [58]:
success.head(2)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
17,19,CCAFS LC-40,1,1952.0,F9 v1.1 B1018,v1.1
18,20,CCAFS LC-40,1,2034.0,F9 FT B1019,FT


In [101]:
app = Dash(__name__)

app.layout=html.Div([
html.H1('SpaceX Launch Records Dashboard',style={'textAlign':'center'}),
    dcc.Dropdown(id='site-dropdown',
                options=[{'label':'All Sites','value':'ALL'}] + [{'label': i ,'value':i}for i in success['Launch Site'].unique()],
                value='ALL'),
    dcc.Graph(id='success-pie-chart'),
    dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=[1000,3000]),
    dcc.Graph(id='success-payload-scatter-chart')
])

@callback(
    Output('success-pie-chart','figure'),
    Input('site-dropdown','value')
)

def first_graph(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(success,names='Launch Site',)
    else:
        selected_data = success[success['Launch Site'] == selected_site]
        selected_success = selected_data['class'].value_counts()
        other_data = success[success['Launch Site'] != selected_site]
        other_success = other_data['class'].value_counts()
        pie_data = pd.DataFrame({
            'Site': ['Selected Site - Success', 'Other Sites - Success'[1]],
            'Counts': [selected_success[1],other_success[1]]
        })
        fig = px.pie(pie_data, values='Counts', names='Site',
                     title=f"Launch Success Comparison for {selected_site} vs Other Sites")
    return fig

@callback(
    Output('success-payload-scatter-chart','figure'),
    Input('payload-slider','value')
)

def second_graph(payload_range):
    filtered_payload = df[(df['Payload Mass (kg)'] >= payload_range[0]) & (df['Payload Mass (kg)'] <= payload_range[1])]
    fig = px.scatter(filtered_payload, x='Payload Mass (kg)', y='class', color='class',
                     labels={"class": "Launch Success"}, title="Launch Success by Payload Mass")
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)